# Filter Control Study

In this notebook we will create several cellwise energy filters based on simulation data for three different energy domains.

We will create and analyze several filters:
- Subtract overall average energy from each cell.
- Subtract average energy by layer.
- Subtract energy weighted by z coordinate.
- Maybe some FFT stuff with time binning.

In [1]:
# Imports ======================================================================
import ROOT as root
import pyLCIO as lcio
import pandas as pd
import numpy as np
from glob import glob
from pprint import pprint
from itertools import zip_longest

# Functionality ================================================================
root.EnableImplicitMT()  # Multithreading

# ==============================================================================

ModuleNotFoundError: No module named 'ROOT'

In [ ]:
# Stylization ==================================================================
root.gStyle.SetOptTitle(0)  # Remove title from hists so you can add a customized one
# ROOT.gStyle.SetTitleTextColor(17)
# ROOT.gStyle.SetTitleAlign(23)
root.gStyle.SetOptStat(0)
root.gStyle.SetCanvasColor(1)
root.gStyle.SetTitleColor(17, "XYZ")
root.gStyle.SetTitleSize(.023, "XYZ")
root.gStyle.SetTitleOffset(1.2, "X")
root.gStyle.SetTitleOffset(1, "Y")
root.gStyle.SetLabelColor(17, "XYZ")
root.gStyle.SetLabelOffset()
root.gStyle.SetNdivisions(103, "XY")
root.gStyle.SetAxisColor(17, "XYZ")
root.gStyle.SetLabelSize(0.02, "XYZ")
root.gStyle.SetPalette(root.kDarkBodyRadiator)
root.gStyle.SetCanvasPreferGL(1)

# ==============================================================================

In [ ]:
# Function for dynamically-scalable file getting.
def get_files(num_files):
    NUM_EVENTS_PER_SIG_FILE = 100
    NUM_EVENTS_PER_BIB_FILE = 10
    files = {
        'SIG' : glob('/scratch/rhillman/data/*_???.parquet'), 
        'BIB' : glob('/data/fmeloni/DataMuC_MuColl10_v0A/v2/recoBIB/nuGun_pT_0_50/*.slcio')
        }
    files['SIG'] = files['SIG'][:num_files]
    files['BIB'] = files['BIB'][:num_files]
    return files

files = get_files(1)
files

{'SIG': [],
 'BIB': ['/data/fmeloni/DataMuC_MuColl10_v0A/v2/recoBIB/nuGun_pT_0_50/nuGun_pT_0_50_reco_1760.slcio']}

In [ ]:
# %%bash
# anajob /data/fmeloni/DataMuC_MuColl10_v0A/v2/recoBIB/nuGun_pT_0_50/nuGun_pT_0_50_reco_1760.slcio

Now that we have a list of available files, we need to be able to parse them. Let's write a function to parse a file and extract relevant information.

We need 2 quantities with a variable threshold (filter threshold).
- Signal efficiency (pf-SIG-energy / tot-SIG-energy)
- BIB efficiency (pf-BIB-energy / tot-BIB-energy)

In [ ]:
def read_file(file, collections):
    reader = lcio.IOIMPL.LCFactory.getInstance().createLCReader(lcio.IO.LCReader.directAccess)
    reader.setReadCollectionNames(collections)
    reader.open(file)
    return reader
    
collections = ['ECalBarrelCollection', 'ECalEndcapCollection']

reader_sig = read_file(files['SIG'][0], collections)
reader_bib = read_file(files['BIB'][0], collections)

IndexError: list index out of range

In [ ]:
def read_event(reader, run_number, event_index):
    event = reader.readEvent(run_number, event_index)
    return event

bib_event = read_event(reader_bib, 0, 1760)
sig_event = read_event(reader_sig, 0, 99606)

In [ ]:
# Create histograms.
nbins = 900
low = -2400
high = 2400
h1 = root.TH2D(name="xy_nofilter", 
               title="xy_nofilter",
               nbinsx=nbins,
               xlow=low,
               xup=high,
               nbinsy=nbins,
               ylow=low,
               yup=high
               )
h2 = root.TH2D(name="xy_simplefilter", 
               title="xy_simplefilter",
               nbinsx=nbins,
               xlow=low,
               xup=high,
               nbinsy=nbins,
               ylow=low,
               yup=high
               )
h3 = root.TH2D(name="xy_layerfilter", 
               title="xy_layerfilter",
               nbinsx=nbins,
               xlow=low,
               xup=high,
               nbinsy=nbins,
               ylow=low,
               yup=high
               )

In [ ]:
# %jsroot on
h1.Clear()

print(bib_event.getCollectionNames())

def get_values(simhit):
    x = simhit.getPosition()[0]
    y = simhit.getPosition()[1]
    energy = simhit.getEnergy()
    return x, y, energy

def fill_hist_with_collection(hist, collection, filter, threshold):
    for simhit in collection:
        x, y, energy = get_values(simhit)
        if filter(energy, threshold): hist.Fill(x, y, energy)
    
def plot_and_filter(collections: list, event: lcio.EVENT.LCEvent, filter, threshold, hist: root.TH2D): 
    collections_list = [event.getCollection(collection) for collection in event.getCollectionNames()]
    print(collections_list)
    for collection in collections_list:
        fill_hist_with_collection(hist, collection, filter, threshold)

def simple_filter(energy, threshold):
    if energy > threshold: 
        return 1
    else: return 0

# def layer_weighted_filter(energy, layer, threshold):


threshold = 0.000#GeV, 0MeV
plot_and_filter(collections, sig_event, simple_filter, threshold, h1)
plot_and_filter(collections, bib_event, simple_filter, threshold, h1)
threshold = 0.017#GeV, 17MeV
plot_and_filter(collections, sig_event, simple_filter, threshold, h2)
plot_and_filter(collections, bib_event, simple_filter, threshold, h2)
# threshold = 0.017#GeV, 17MeV
# plot_and_filter(collections, bib_event, layer_weighted_filter, threshold, h3)
# plot_and_filter(collections, sig_event, simple_filter, threshold, h3)

    
# Create a new canvas with two pads
c = root.TCanvas("c1", "2-D Histograms", 400, 400)
c.SetBatch()
c.Divide(2, 2)  # Divide the canvas into two pads
c.GetPad(1).SetLogz()
c.GetPad(2).SetLogz()

# First pad
c.cd(1)
h1.GetYaxis().CenterTitle()
h1.GetYaxis().RotateTitle()
h1.GetXaxis().CenterTitle()
h1.GetZaxis().SetTitle("#bf{Energy} #scale[.8]{#color[13]{#[]{Ge#kern[-.115]{V}#kern[.2]{/}#it{c}^{2}}}}")
h1.GetZaxis().SetTitleOffset(0.5)
h1.GetZaxis().RotateTitle()
h1.Draw('COL Z PLC')

# Second pad
c.cd(2)
h2.GetYaxis().CenterTitle()
h2.GetYaxis().RotateTitle()
h2.GetXaxis().CenterTitle()
h2.GetZaxis().SetTitle("#bf{Energy} #scale[.8]{#color[13]{#[]{Ge#kern[-.115]{V}#kern[.2]{/}#it{c}^{2}}}}")
h2.GetZaxis().SetTitleOffset(0.5)
h2.GetZaxis().RotateTitle()
h2.Draw('COL Z PLC')

c.Draw()